# Vytvorenie regresného modelu: príprava a vizualizácia dát

## **Lineárna regresia pre tekvice - Lekcia 2**
#### Úvod

Teraz, keď máte pripravené nástroje na budovanie modelov strojového učenia pomocou Tidymodels a Tidyverse, ste pripravení začať klásť otázky o svojich dátach. Pri práci s dátami a aplikovaní riešení strojového učenia je veľmi dôležité vedieť, ako položiť správnu otázku, aby ste mohli plne využiť potenciál svojho datasetu.

V tejto lekcii sa naučíte:

-   Ako pripraviť svoje dáta na budovanie modelov.

-   Ako používať `ggplot2` na vizualizáciu dát.

Otázka, na ktorú potrebujete odpoveď, určí, aký typ algoritmov strojového učenia budete používať. Kvalita odpovede, ktorú dostanete, bude výrazne závisieť od povahy vašich dát.

Pozrime sa na to prostredníctvom praktického cvičenia.


<p >
   <img src="../../images/unruly_data.jpg"
   width="700"/>
   <figcaption>Ilustrácia od @allison_horst</figcaption>


<!--![Ilustrácia od \@allison_horst](../../../../../../2-Regression/2-Data/images/unruly_data.jpg)<br>Ilustrácia od \@allison_horst-->


## 1. Importovanie údajov o tekviciach a privolanie Tidyverse

Na spracovanie tejto lekcie budeme potrebovať nasledujúce balíky:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) je [kolekcia balíkov pre R](https://www.tidyverse.org/packages), ktorá je navrhnutá tak, aby robila dátovú vedu rýchlejšou, jednoduchšou a zábavnejšou!

Môžete ich nainštalovať pomocou:

`install.packages(c("tidyverse"))`

Nasledujúci skript skontroluje, či máte nainštalované balíky potrebné na dokončenie tohto modulu, a v prípade, že niektoré chýbajú, ich pre vás nainštaluje.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))
pacman::p_load(tidyverse)

Teraz si spustime niektoré balíčky a načítajme [dáta](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/data/US-pumpkins.csv) poskytnuté pre túto lekciu!


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n =50)

Rýchly pohľad pomocou `glimpse()` okamžite ukáže, že existujú prázdne hodnoty a mix reťazcov (`chr`) a číselných údajov (`dbl`). `Date` je typu znakový reťazec a je tu aj zvláštny stĺpec s názvom `Package`, kde sú údaje zmiešané medzi `sacks`, `bins` a inými hodnotami. Údaje sú, úprimne povedané, trochu chaotické 😤.

V skutočnosti nie je veľmi bežné dostať dataset, ktorý je úplne pripravený na použitie na vytvorenie ML modelu hneď po rozbalení. Ale nebojte sa, v tejto lekcii sa naučíte, ako pripraviť surový dataset pomocou štandardných knižníc v R 🧑‍🔧. Taktiež sa naučíte rôzne techniky na vizualizáciu údajov. 📈📊
<br>

> Pripomenutie: Operátor pipe (`%>%`) vykonáva operácie v logickej postupnosti tým, že posúva objekt ďalej do funkcie alebo výrazu. Môžete si operátor pipe predstaviť ako výraz „a potom“ vo vašom kóde.


## 2. Kontrola chýbajúcich údajov

Jedným z najbežnejších problémov, s ktorými sa dátoví analytici musia vysporiadať, sú neúplné alebo chýbajúce údaje. R reprezentuje chýbajúce alebo neznáme hodnoty pomocou špeciálnej hodnoty: `NA` (Not Available).

Ako teda zistíme, že dátový rámec obsahuje chýbajúce hodnoty?
<br>
-   Jedným z priamych spôsobov je použitie základnej funkcie R `anyNA`, ktorá vracia logické objekty `TRUE` alebo `FALSE`.


In [ ]:
pumpkins %>% 
  anyNA()

Zdá sa, že niektoré údaje chýbajú! To je dobré miesto, kde začať.

-   Ďalším spôsobom by bolo použiť funkciu `is.na()`, ktorá označuje, ktoré jednotlivé prvky stĺpcov chýbajú pomocou logickej hodnoty `TRUE`.


In [ ]:
pumpkins %>% 
  is.na() %>% 
  head(n = 7)

Dobre, práca je hotová, ale s takým veľkým dátovým rámcom by bolo neefektívne a prakticky nemožné kontrolovať všetky riadky a stĺpce jednotlivo😴.

-   Intuitívnejší spôsob by bol vypočítať súčet chýbajúcich hodnôt pre každý stĺpec:


In [ ]:
pumpkins %>% 
  is.na() %>% 
  colSums()

Oveľa lepšie! Chýbajú niektoré údaje, ale možno to nebude mať vplyv na danú úlohu. Uvidíme, aké ďalšie analýzy prinesú výsledky.

> Okrem skvelých balíkov a funkcií má R veľmi dobrú dokumentáciu. Napríklad použite `help(colSums)` alebo `?colSums`, aby ste sa dozvedeli viac o funkcii.


## 3. Dplyr: Gramatika manipulácie s dátami

<p >
   <img src="../../images/dplyr_wrangling.png"
   width="569"/>
   <figcaption>Ilustrácia od @allison_horst</figcaption>


<!--![Ilustrácia od \@allison_horst](../../../../../../2-Regression/2-Data/images/dplyr_wrangling.png)<br/>Ilustrácia od \@allison_horst-->


[`dplyr`](https://dplyr.tidyverse.org/), balík v Tidyverse, je gramatika manipulácie s dátami, ktorá poskytuje konzistentnú sadu slovies, ktoré vám pomôžu vyriešiť najbežnejšie výzvy pri manipulácii s dátami. V tejto sekcii preskúmame niektoré zo slovies dplyr!


#### dplyr::select()

`select()` je funkcia v balíku `dplyr`, ktorá vám pomáha vybrať stĺpce na ponechanie alebo vylúčenie.

Aby ste uľahčili prácu s vaším dátovým rámcom, odstráňte niekoľko jeho stĺpcov pomocou `select()`, pričom ponechajte iba tie, ktoré potrebujete.

Napríklad v tomto cvičení bude naša analýza zahŕňať stĺpce `Package`, `Low Price`, `High Price` a `Date`. Vyberme tieto stĺpce.


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(Package, `Low Price`, `High Price`, Date)


# Print data set
pumpkins %>% 
  slice_head(n = 5)

#### dplyr::mutate()

`mutate()` je funkcia v balíku `dplyr`, ktorá vám umožňuje vytvárať alebo upravovať stĺpce, pričom zachováva existujúce stĺpce.

Všeobecná štruktúra funkcie mutate je:

`data %>%   mutate(new_column_name = what_it_contains)`

Poďme si vyskúšať `mutate` na stĺpci `Date` vykonaním nasledujúcich operácií:

1.  Konvertovať dátumy (aktuálne typu character) na formát mesiaca (ide o americké dátumy, takže formát je `MM/DD/YYYY`).

2.  Extrahovať mesiac z dátumov do nového stĺpca.

V R balík [lubridate](https://lubridate.tidyverse.org/) uľahčuje prácu s dátumovo-časovými údajmi. Takže použijeme `dplyr::mutate()`, `lubridate::mdy()`, `lubridate::month()` a pozrieme sa, ako dosiahnuť vyššie uvedené ciele. Stĺpec Date môžeme vynechať, pretože ho už nebudeme potrebovať v ďalších operáciách.


In [ ]:
# Load lubridate
library(lubridate)

pumpkins <- pumpkins %>% 
  # Convert the Date column to a date object
  mutate(Date = mdy(Date)) %>% 
  # Extract month from Date
  mutate(Month = month(Date)) %>% 
  # Drop Date column
  select(-Date)

# View the first few rows
pumpkins %>% 
  slice_head(n = 7)

Hurá! 🤩

Ďalej vytvorme nový stĺpec `Price`, ktorý bude predstavovať priemernú cenu tekvice. Teraz vypočítajme priemer stĺpcov `Low Price` a `High Price`, aby sme naplnili nový stĺpec Price.  
<br>


In [ ]:
# Create a new column Price
pumpkins <- pumpkins %>% 
  mutate(Price = (`Low Price` + `High Price`)/2)

# View the first few rows of the data
pumpkins %>% 
  slice_head(n = 5)

Yeees!💪

„Ale počkaj!“, poviete si po rýchlom prehliadnutí celého dátového súboru pomocou `View(pumpkins)`, „Tu je niečo zvláštne!“🤔

Ak sa pozriete na stĺpec `Package`, tekvice sa predávajú v rôznych konfiguráciách. Niektoré sa predávajú v mierach `1 1/9 bushel`, niektoré v mierach `1/2 bushel`, niektoré na kusy, niektoré na váhu a niektoré vo veľkých škatuliach s rôznymi šírkami.

Poďme si to overiť:


In [ ]:
# Verify the distinct observations in Package column
pumpkins %>% 
  distinct(Package)

Úžasné!👏

Zdá sa, že tekvice je veľmi ťažké vážiť konzistentne, takže ich vyfiltrujme tak, že vyberieme iba tekvice, ktoré obsahujú reťazec *bushel* v stĺpci `Package`, a uložme to do nového dátového rámca `new_pumpkins`.


#### dplyr::filter() a stringr::str_detect()

[`dplyr::filter()`](https://dplyr.tidyverse.org/reference/filter.html): vytvára podmnožinu údajov obsahujúcu iba **riadky**, ktoré spĺňajú vaše podmienky, v tomto prípade tekvice so slovom *bushel* v stĺpci `Package`.

[stringr::str_detect()](https://stringr.tidyverse.org/reference/str_detect.html): zisťuje prítomnosť alebo neprítomnosť vzoru v reťazci.

Balík [`stringr`](https://github.com/tidyverse/stringr) poskytuje jednoduché funkcie pre bežné operácie s reťazcami.


In [ ]:
# Retain only pumpkins with "bushel"
new_pumpkins <- pumpkins %>% 
       filter(str_detect(Package, "bushel"))

# Get the dimensions of the new data
dim(new_pumpkins)

# View a few rows of the new data
new_pumpkins %>% 
  slice_head(n = 5)

Môžete vidieť, že sme zúžili výber na približne 415 riadkov údajov obsahujúcich tekvice na veľké množstvo.🤩  
<br>


#### dplyr::case_when()

**Ale počkajte! Ešte je tu jedna vec, ktorú treba urobiť**

Všimli ste si, že množstvo v bušloch sa líši v jednotlivých riadkoch? Musíte normalizovať ceny tak, aby ste zobrazili cenu za bušel, nie za 1 1/9 alebo 1/2 bušela. Je čas na trochu matematiky, aby ste to štandardizovali.

Použijeme funkciu [`case_when()`](https://dplyr.tidyverse.org/reference/case_when.html) na *mutáciu* stĺpca Price v závislosti od určitých podmienok. `case_when` umožňuje vektorovo spracovať viacero `if_else()` vyjadrení.


In [ ]:
# Convert the price if the Package contains fractional bushel values
new_pumpkins <- new_pumpkins %>% 
  mutate(Price = case_when(
    str_detect(Package, "1 1/9") ~ Price/(1 + 1/9),
    str_detect(Package, "1/2") ~ Price/(1/2),
    TRUE ~ Price))

# View the first few rows of the data
new_pumpkins %>% 
  slice_head(n = 30)

Teraz môžeme analyzovať cenu za jednotku na základe ich merania v bušloch. Celá táto štúdia o bušloch tekvíc však ukazuje, aké veľmi `dôležité` je `pochopiť povahu vašich údajov`!

> ✅ Podľa [The Spruce Eats](https://www.thespruceeats.com/how-much-is-a-bushel-1389308) hmotnosť bušľa závisí od typu produktu, pretože ide o objemové meranie. "Bušel paradajok, napríklad, by mal vážiť 56 libier... Listy a zelené rastliny zaberajú viac miesta s menšou hmotnosťou, takže bušel špenátu váži iba 20 libier." Je to všetko dosť komplikované! Nezaoberajme sa konverziou bušlov na libry, namiesto toho stanovme cenu za bušel. Celá táto štúdia o bušloch tekvíc však ukazuje, aké veľmi dôležité je pochopiť povahu vašich údajov!
>
> ✅ Všimli ste si, že tekvice predávané na pol bušľa sú veľmi drahé? Dokážete zistiť prečo? Tip: malé tekvice sú oveľa drahšie ako veľké, pravdepodobne preto, že ich je oveľa viac na bušel, vzhľadom na nevyužitý priestor, ktorý zaberá jedna veľká dutá tekvica na koláč.


DÔLEŽITÉ PRAVIDLÁ:  
1. NEPRIDÁVAJTE '''markdown alebo akékoľvek iné značky okolo prekladu  
2. Uistite sa, že preklad neznie príliš doslovne  
3. Preložte aj komentáre  
4. Tento súbor je napísaný vo formáte Markdown - nezaobchádzajte s ním ako s XML alebo HTML  
5. Neprekladajte:  
   - [!NOTE], [!WARNING], [!TIP], [!IMPORTANT], [!CAUTION]  
   - Názvy premenných, funkcií, tried  
   - Zástupné symboly ako @@INLINE_CODE_x@@ alebo @@CODE_BLOCK_x@@  
   - URL alebo cesty  
6. Zachovajte všetko pôvodné formátovanie Markdownu  
7. Vráťte IBA preložený obsah bez akýchkoľvek dodatočných značiek alebo formátovania  

A teraz nakoniec, len pre dobrodružstvo 💁‍♀️, presuňme stĺpec Month na prvú pozíciu, teda `pred` stĺpec `Package`.  

Na zmenu pozícií stĺpcov sa používa `dplyr::relocate()`.  


In [ ]:
# Create a new data frame new_pumpkins
new_pumpkins <- new_pumpkins %>% 
  relocate(Month, .before = Package)

new_pumpkins %>% 
  slice_head(n = 7)

Skvelá práca!👌 Teraz máte čistý, uprataný dataset, na ktorom môžete postaviť svoj nový regresný model!  
<br>


## 4. Vizualizácia dát s ggplot2

<p >
   <img src="../../images/data-visualization.png"
   width="600"/>
   <figcaption>Infografika od Dasani Madipalli</figcaption>


<!--![Infografika od Dasani Madipalli](../../../../../../2-Regression/2-Data/images/data-visualization.png){width="600"}-->

Existuje *múdre* príslovie, ktoré hovorí:

> "Jednoduchý graf priniesol analytikovi dát viac informácií než akékoľvek iné zariadenie." --- John Tukey

Úlohou dátového vedca je ukázať kvalitu a charakter dát, s ktorými pracuje. Na tento účel často vytvára zaujímavé vizualizácie, ako sú grafy, diagramy a tabuľky, ktoré zobrazujú rôzne aspekty dát. Týmto spôsobom dokáže vizuálne ukázať vzťahy a medzery, ktoré by inak bolo ťažké odhaliť.

Vizualizácie môžu tiež pomôcť určiť, ktorá technika strojového učenia je pre dané dáta najvhodnejšia. Napríklad bodový graf, ktorý sa zdá sledovať líniu, naznačuje, že dáta sú vhodným kandidátom na cvičenie s lineárnou regresiou.

R ponúka niekoľko systémov na tvorbu grafov, ale [`ggplot2`](https://ggplot2.tidyverse.org/index.html) je jedným z najelegantnejších a najuniverzálnejších. `ggplot2` vám umožňuje vytvárať grafy **kombinovaním nezávislých komponentov**.

Začnime jednoduchým bodovým grafom pre stĺpce Price a Month.

V tomto prípade začneme s [`ggplot()`](https://ggplot2.tidyverse.org/reference/ggplot.html), poskytneme dataset a estetické mapovanie (pomocou [`aes()`](https://ggplot2.tidyverse.org/reference/aes.html)), a potom pridáme vrstvy (ako [`geom_point()`](https://ggplot2.tidyverse.org/reference/geom_point.html)) pre bodové grafy.


In [ ]:
# Set a theme for the plots
theme_set(theme_light())

# Create a scatter plot
p <- ggplot(data = new_pumpkins, aes(x = Price, y = Month))
p + geom_point()

Je to užitočný graf 🤷? Prekvapilo vás na ňom niečo?

Nie je obzvlášť užitočný, pretože iba zobrazuje vaše údaje ako rozptyl bodov v danom mesiaci.  
<br>


### **Ako to urobiť užitočným?**

Aby grafy zobrazovali užitočné údaje, zvyčajne je potrebné údaje nejako zoskupiť. Napríklad v našom prípade by zistenie priemernej ceny tekvíc za každý mesiac poskytlo viac informácií o základných vzoroch v našich údajoch. To nás privádza k ďalšiemu preletu nad **dplyr**:

#### `dplyr::group_by() %>% summarize()`

Zoskupenú agregáciu v R je možné jednoducho vypočítať pomocou

`dplyr::group_by() %>% summarize()`

-   `dplyr::group_by()` mení jednotku analýzy z celého datasetu na jednotlivé skupiny, napríklad podľa mesiacov.

-   `dplyr::summarize()` vytvára nový dátový rámec s jedným stĺpcom pre každú zoskupovaciu premennú a jedným stĺpcom pre každú štatistiku, ktorú ste špecifikovali.

Napríklad môžeme použiť `dplyr::group_by() %>% summarize()` na zoskupenie tekvíc do skupín na základe stĺpca **Month** a potom vypočítať **priemernú cenu** za každý mesiac.


In [ ]:
# Find the average price of pumpkins per month
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price))

Stručne!✨

Kategorické prvky, ako sú mesiace, sú lepšie znázornené pomocou stĺpcového grafu 📊. Vrstvy zodpovedné za stĺpcové grafy sú `geom_bar()` a `geom_col()`. Viac informácií nájdete v `?geom_bar`.

Poďme si jeden vytvoriť!


In [ ]:
# Find the average price of pumpkins per month then plot a bar chart
new_pumpkins %>%
  group_by(Month) %>% 
  summarise(mean_price = mean(Price)) %>% 
  ggplot(aes(x = Month, y = mean_price)) +
  geom_col(fill = "midnightblue", alpha = 0.7) +
  ylab("Pumpkin Price")

🤩🤩 Toto je užitočnejšia vizualizácia dát! Zdá sa, že najvyššia cena za tekvice sa vyskytuje v septembri a októbri. Zodpovedá to vašim očakávaniam? Prečo áno alebo prečo nie?

Gratulujeme k dokončeniu druhej lekcie 👏! Pripravili ste svoje dáta na vytváranie modelov a potom ste odhalili ďalšie poznatky pomocou vizualizácií!



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby na automatický preklad [Co-op Translator](https://github.com/Azure/co-op-translator). Aj keď sa snažíme o presnosť, upozorňujeme, že automatické preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre dôležité informácie odporúčame profesionálny ľudský preklad. Nezodpovedáme za žiadne nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
